# 영화 관객수 예측 모델
감독, 이름, 상영등급, 스태프 수 등의 정보를 이용하여 영화 관객 수를 예측하는 모델을 만들어 보는 것

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Bigdata_project_moviecatcher/Modelling

/content/drive/MyDrive/Bigdata_project_moviecatcher/Modelling


## 1. 라이브러리 및 Data set 불러오기

In [2]:
import pandas as pd
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import ElasticNet, ElasticNetCV # 두 개 구분
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [43]:
# csv로 변환
df = pd.read_excel('data/Preprocessed_Data_19_20.xlsx')
#print(df)
df.to_csv('data/Preprocessed_Data_19_20.csv',index=True, sep='\t')
print(df)

    Unnamed: 0       movie_name  ...  abs_mean_score   actor_power
0            0             극한직업  ...        1.019999  4.539712e+06
1            1       어벤져스: 엔드게임  ...        2.126920  9.367612e+06
2            2           겨울왕국 2  ...        1.154460  9.751492e+06
3            3              알라딘  ...        1.495438  6.752496e+06
4            4              기생충  ...        2.085190  5.109347e+06
..         ...              ...  ...             ...           ...
89          89        프리즌 이스케이프  ...        0.791509  8.232217e+04
90          90               검객  ...        3.028084  1.902370e+05
91          91               조제  ...        2.439042  1.335327e+06
92          92           사라진 시간  ...        3.225536  1.872178e+06
93          93  밤쉘: 세상을 바꾼 폭탄선언  ...        4.372499  5.237502e+05

[94 rows x 12 columns]


In [61]:
df = pd.read_table('data/Preprocessed_Data_19_20.csv')
print(df)

df1 = df.iloc[4:94,:] 
#df1 = df1[['movie_name','actor_power']]
df2 = df.iloc[0:4,:]
df1.to_csv('data/movies_train.csv',sep=',')
df2.to_csv('data/movies_test.csv',sep=',')

    Unnamed: 0  Unnamed: 0.1  ... abs_mean_score   actor_power
0            0             0  ...       1.019999  4.539712e+06
1            1             1  ...       2.126920  9.367612e+06
2            2             2  ...       1.154460  9.751492e+06
3            3             3  ...       1.495438  6.752496e+06
4            4             4  ...       2.085190  5.109347e+06
..         ...           ...  ...            ...           ...
89          89            89  ...       0.791509  8.232217e+04
90          90            90  ...       3.028084  1.902370e+05
91          91            91  ...       2.439042  1.335327e+06
92          92            92  ...       3.225536  1.872178e+06
93          93            93  ...       4.372499  5.237502e+05

[94 rows x 13 columns]


In [62]:
# 데이터 불러오기
train = pd.read_csv('data/movies_train.csv')
test = pd.read_csv('data/movies_test.csv')
submission = pd.read_csv('data/submission.csv')

In [63]:
test

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,movie_name,audience,nation,distribution_company,genre,running_time,age,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power
0,0,0,0,극한직업,16265618,한국,(주)씨제이이엔엠,코미디,111,15세이상관람가,9518,616945.8,1.019999,4.539712e+06
1,1,1,1,어벤져스: 엔드게임,13934592,미국,월트디즈니컴퍼니코리아 유한책임회사,액션,180,12세이상관람가,26299,3047574.7,2.126920,9.367612e+06
2,2,2,2,겨울왕국 2,27495584,미국,월트디즈니컴퍼니코리아 유한책임회사,애니메이션,103,전체관람가,20661,9814174.2,1.154460,9.751492e+06
3,3,3,3,알라딘,25447554,미국,월트디즈니컴퍼니코리아 유한책임회사,어드벤처,127,전체관람가,6934,4894125.2,1.495438,6.752496e+06


In [64]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,movie_name,audience,nation,distribution_company,genre,running_time,age,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power
0,4,4,4,기생충,20626290,한국,(주)씨제이이엔엠,드라마,131,15세이상관람가,10215,1974417.4,2.085190,5.109347e+06
1,5,5,5,엑시트,9426011,한국,(주)씨제이이엔엠,코미디,103,12세이상관람가,2198,408436.3,1.486567,5.557060e+06
2,6,6,6,스파이더맨: 파 프롬 홈,8021145,미국,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,액션,129,12세이상관람가,7007,647694.1,1.496247,3.854988e+06
3,7,7,7,백두산,16505818,한국,(주)씨제이이엔엠,어드벤처,128,12세이상관람가,13381,674718.9,2.728461,6.686322e+06
4,8,8,8,캡틴 마블,5802810,미국,월트디즈니컴퍼니코리아 유한책임회사,액션,123,12세이상관람가,10251,813026.7,1.805130,2.364987e+06


In [20]:
import platform

platform.system()

'Linux'

In [ ]:
cd ../

/


In [ ]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
    print('Mac version')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    print('Windows version')
elif platform.system() == 'Linux':
    path = "/usr/share/fonts/NanumFont/NanumGothicBold.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
    print('Linux version')
else:
    print('Unknown system... sorry~~~~')


FileNotFoundError: ignored

## 3. 데이터 전처리
## Data Cleansing & Pre-Processing  

### isna함수는 결측치 여부를 확인

In [65]:
train.isna().head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,movie_name,audience,nation,distribution_company,genre,running_time,age,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False


이렇게 보면 결측치를 확인하기 어렵다. 여기서 sum함수를 이용해 확인해보자.

In [66]:
train.isna().sum()

Unnamed: 0               0
Unnamed: 0.1             0
Unnamed: 0.1.1           0
movie_name               0
audience                 0
nation                   0
distribution_company     0
genre                    0
running_time             0
age                      0
youtube_comment_count    0
youtube_view_mean        0
abs_mean_score           0
actor_power              0
dtype: int64

In [23]:
test.isna().head()

,"Unnamed: 0,movie_name,audience,nation,distribution_company,genre,running_time,age,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power"
0,False
1,False
2,False


In [26]:
test.isna().sum()

Unnamed: 0,movie_name,audience,nation,distribution_company,genre,running_time,age,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power    0
dtype: int64

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [27]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

random_state=777은 어느 pc에서도 결과값이 같게 나오게 고정시켜준다고 이해를 하자. n_estimators은 순차적으로 만드는 모델을 1000번 반복해서 만들겟다는 뜻

In [70]:
#features = ['distribution_company','genre','running_time','age','nation','youtube_comment_count','youtube_view_mean','abs_mean_score','actor_power']
features = ['running_time','youtube_comment_count','youtube_view_mean','abs_mean_score','actor_power']
target = ['audience']

이제 독립변수(features)와 종속변수(target)를 만들어주자.

In [71]:
X_train, X_test, y_train = train[features], test[features], train[target]

X,y를 이제 tarin,test에 변수들을 저장해주도록 하자. 

In [72]:
model.fit(X_train, y_train)

LGBMRegressor(n_estimators=1000, random_state=777)

#### LGBM 모델을 학습시켜주기

In [73]:
singleLGBM = submission.copy()

In [74]:
singleLGBM.head()

,movie_name,audience
0,극한직업,0
1,어벤져서: 엔드게임,0
2,겨울왕국 2,0
3,알라딘,0


In [75]:
singleLGBM['audience'] = model.predict(X_test)

In [76]:
singleLGBM.head()

,movie_name,audience
0,극한직업,6.211123e+06
1,어벤져서: 엔드게임,1.670796e+07
2,겨울왕국 2,7.339427e+06
3,알라딘,9.139713e+06


In [77]:
6.211123e+06

6211123.0

In [3]:
mean_squared_error(1,2)

TypeError: ignored

많이 잘못 됬다는것을 볼 수 있다. 관객수가 음수값이 나올수가 없는데 음수가 나오는것을 확인할수있다. 이 모델은 성능이 별로 안좋은거 같다. 다른 모델로 넘어가보도록 하자.

## 다음 모델을 사용해보기. -> 글씨 형식 다 수치형 데이터로 변환해주기

In [82]:
features

['running_time',
 'youtube_comment_count',
 'youtube_view_mean',
 'abs_mean_score',
 'actor_power']

- 글자로 된 데이터 숫자로 변환하여 사용하기

In [78]:
train.genre

0        드라마
1        코미디
2         액션
3       어드벤처
4         액션
       ...  
85      어드벤처
86        액션
87    멜로/로맨스
88      미스터리
89       드라마
Name: genre, Length: 90, dtype: object

In [79]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() 
train['genre'] = le.fit_transform(train['genre'])

sklearn에서 제공하는 labelEncoder를 활용해서 문자열을 숫자로 변환해보자.

In [80]:
train['genre']

0      1
1     11
2      8
3      9
4      8
      ..
85     9
86     8
87     2
88     3
89     1
Name: genre, Length: 90, dtype: int64

숫자로 변환되있는걸 확인 할 수 있다. test셋도 변환해주자.

In [81]:
test['genre'] = le.transform(test['genre'])

위 fit_transfirm인데 test셋은 transform으로만 변환하였다. test셋에 또 fit(훈련하다)을 넣어주면 결과 값이 틀려질수 있으니 transform만 해야된다.

In [ ]:
test['genre']

0       2
1       5
2       4
3      10
4       1
       ..
238     4
239     3
240     2
241     4
242     5
Name: genre, Length: 243, dtype: int64

test셋도 변환될걸 확인 할 수 있다.

In [83]:
features = ['genre','running_time','youtube_comment_count','youtube_view_mean','abs_mean_score','actor_power']


features에 전처리된 dir_prev_bfnum와 문자열을 숫자로 변환한 genre를 추가해주자.

In [84]:
X_train, X_test, y_train = train[features], test[features], train[target]

컬럼이 2개 추가되어 위와 같이 세팅을 해주면 된다.

#### simple LGBM으로 다시 학습

In [87]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

In [88]:
model.fit(X_train, y_train)

LGBMRegressor(n_estimators=1000, random_state=777)

In [89]:
singleLGBM = submission.copy()

In [90]:
singleLGBM['audience'] = model.predict(X_test)

In [91]:
singleLGBM

,movie_name,audience
0,극한직업,5.960190e+06
1,어벤져서: 엔드게임,1.318630e+07
2,겨울왕국 2,6.458905e+06
3,알라딘,8.771846e+06


In [93]:
test['audience']

0    16265618
1    13934592
2    27495584
3    25447554
Name: audience, dtype: int64

In [ ]:
feLightGBM.head()

,title,box_off_num
0,용서는 없다,3.395493e+06
1,아빠가 여자를 좋아해,8.235439e+05
2,하모니,1.162055e+06
3,의형제,2.184689e+06
4,평행 이론,8.093288e+05


In [ ]:
feLightGBM['box_off_num'] = np.mean(preds, axis = 0)

In [ ]:
feLightGBM = submission.copy()

In [ ]:
preds = []
for model in models:
    preds.append(model.predict(X_test))
len(preds)

5

In [85]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

NameError: ignored

In [86]:
X_test.head()

,genre,running_time,youtube_comment_count,youtube_view_mean,abs_mean_score,actor_power
0,11,111,9518,616945.8,1.019999,4.539712e+06
1,8,180,26299,3047574.7,2.126920,9.367612e+06
2,7,103,20661,9814174.2,1.154460,9.751492e+06
3,9,127,6934,4894125.2,1.495438,6.752496e+06


## Grid Search 모델 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

In [ ]:
params = {
    'learning_rate': [0.1, 0.01, 0.003],
    'min_child_samples': [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid=params,
                  scoring= 'neg_mean_squared_error',
                  cv = k_fold)

params에서 learning_rate는 모델링을 하는 간격으로, 값이 적을수록 점점 더 미세하게 모델의 변화가 이루어진다로 생각하면 된다.
 
scoring을 rmse로 한 이후는 현재 이 대회의 평가지표가 rmse값이기 때문에 이렇게 설정하였습니다.

In [ ]:
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             estimator=LGBMRegressor(n_estimators=1000, random_state=777),
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                         'min_child_samples': [20, 30]},
             scoring='neg_mean_squared_error')

In [ ]:
gs.best_params_

{'learning_rate': 0.003, 'min_child_samples': 30}

In [ ]:
gslgbm.to_csv('gslgbm.csv', index=False)